In [2]:
import pandas as pd 
import numpy as np
%matplotlib inline
import re
from glob import glob
import re
import nltk
from scipy.linalg import eigh, norm
import plotly_express as px
from sklearn.decomposition import PCA
import seaborn as sns
from gensim.models import word2vec
from sklearn.manifold import TSNE
from tqdm import tqdm
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
epub_dir = 'epubs'

C:\Users\maria\AppData\Local\Temp\ipykernel_9512\3178743543.py:24: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import os
os.chdir("C:/Users/maria/OneDrive/Desktop/UVA Data Science/ETA/Final_Project")

In [4]:
df_eliot = pd.read_csv('145.txt', sep='\t', header = None, names=['text'])
df_shelley = pd.read_csv('18247.txt', sep='\t', header = None, names=['text'])
df_woolf = pd.read_csv('144.txt', sep='\t', header = None, names=['text'])

**Conversion**

In [5]:
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
chaps = "First|Second|Third|Fourth|Fifth|Sixth"
chap_pats = {
145: {
    'start_line': 24,
    'end_line': 28115,
    'chapter': re.compile('^\s*CHAPTER\s+{}.\s*$'.format(roman))
    },
18247: {
    'start_line': 22,
    'end_line': 15117,
    'chapter': re.compile('^\s*CHAPTER\s+{}.\s*$'.format(roman))
    },
144: {
    'start_line': 19,
    'end_line': 12223,
    'chapter': re.compile('^\s*CHAPTER\s+{}.\s*$'.format(roman))
    },
}

In [6]:
def acquire_epubs(epub_list, chap_pats, OHCO=OHCO):
    my_lib = []
    my_doc = []

    for epub_file in epub_list:
        # Get PG ID from filename
        book_id = int(epub_file.split('.')[0])
        print("BOOK ID", book_id)

        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = book_id

        # FIX CHARACTERS TO IMPROVE TOKENIZATION
        df.line_str = df.line_str.str.replace('—', ' — ')
        df.line_str = df.line_str.str.replace('-', ' - ')

        # Get book title and put into LIB table -- note problems, though
        book_title = re.sub(r"The Project Gutenberg eBook( of|,) ", "", df.loc[0].line_str, flags=re.IGNORECASE)
        book_title = re.sub(r"Project Gutenberg's ", "", book_title, flags=re.IGNORECASE)

        # Remove cruft
        a = chap_pats[book_id]['start_line'] - 1
        b = chap_pats[book_id]['end_line'] + 1
        df = df.iloc[a:b]

        # Chunk by chapter
        chap_lines = df.line_str.str.match(chap_pats[book_id]['chapter'])
        chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
        df.loc[chap_lines, 'chap_num'] = chap_nums
        df.chap_num = df.chap_num.ffill()

        # Clean up
        df = df[~df.chap_num.isna()] # Remove chapter heading lines
        df = df.loc[~chap_lines] # Remove everything before Chapter 1
        df['chap_num'] = df['chap_num'].astype('int')

        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

        # Split into paragraphs
        df = df['line_str'].astype(str).str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].astype(str).str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].astype(str).str.match(r'^\s*$')] # Remove empty paragraphs

        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_lib.append((book_id, book_title, epub_file))
        my_doc.append(df)

    docs = pd.concat(my_doc)
    library = pd.DataFrame(my_lib, columns=['book_id', 'book_title', 'book_file']).set_index('book_id')
    print("Done.")
    return library, docs


In [7]:
epubs = [epub for epub in sorted(glob('*.txt'))]
LIB, DOC = acquire_epubs(epubs, chap_pats)

BOOK ID 144
BOOK ID 145
BOOK ID 18247
Done.


C:\Users\maria\AppData\Local\Temp\ipykernel_9512\4268693771.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['para_str'] = df['para_str'].astype(str).str.replace(r'\n', ' ').str.strip()
C:\Users\maria\AppData\Local\Temp\ipykernel_9512\4268693771.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['para_str'] = df['para_str'].astype(str).str.replace(r'\n', ' ').str.strip()
C:\Users\maria\AppData\Local\Temp\ipykernel_9512\4268693771.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['para_str'] = df['para_str'].astype(str).str.replace(r'\n', ' ').str.strip()


In [8]:
DOC.sample(10)

para_str
book_id chap_num para_num                                                   
145     92       40                               “Not high - flown enough?”
144     34       161       “They have such excellent biscuits here,” she ...
        27       111                        “I’m an old - fashioned father.”
        46       37          The clock here struck twelve instead of eleven.
        27       123       “No, no,” laughed Willoughby, “the monsters of...
145     116      31        When he was gone, Dorothea’s tears gushed fort...
144     33       35        Creeping on, they found that the next window r...
        30       9         “A grand fellow, Shakespeare,” he said, replac...
145     94       8         “What is the matter with Casaubon? I see no ha...
        149      17        Mr. Farebrother noticed that Lydgate seemed bo...

In [9]:
# split paragraphs into sentences
dfs = DOC['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})

dfs.index.names = OHCO[:4]

dfs = dfs[~dfs['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [10]:
# split into tokens
dft = dfs['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token_str'})
    
dft.index.names = OHCO[:5]
dft

token_str
book_id chap_num para_num sent_num token_num          
144     27       1        0        0           CHAPTER
                                   1                 I
                 2        0        0                As
                                   1               the
                                   2           streets
...                                                ...
18247   58       13       3        1               let
                                   2                me
                                   3             dwell
                                   4                on
                                   5              your

[578422 rows x 1 columns]

**Tokenize and Annotate**

In [11]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x))) # Discards stuff in between
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [12]:
TOKEN = tokenize(DOC, ws=True)

In [30]:
TOKEN.to_csv("TOKEN_final.csv")

In [29]:
TOKEN.head()

pos_tuple  pos token_str
book_id chap_num para_num sent_num token_num                               
144     27       1        0        0           (CHAPTER, NN)   NN   CHAPTER
                                   1                (I, PRP)  PRP         I
                 2        0        0                (As, IN)   IN        As
                                   1               (the, DT)   DT       the
                                   2          (streets, NNS)  NNS   streets

In [ ]:
TOKEN[TOKEN.pos.str.match('^JJ')]

**Extract vocabulary from the Token table**

In [31]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

C:\Users\maria\AppData\Local\Temp\ipykernel_10016\1858674674.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')


In [32]:
VOCAB = TOKEN.term_str.value_counts()\
    .to_frame()\
    .rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index()\
    .reset_index()\
    .rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

# add stopwords to vocab
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [33]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
VOCAB[VOCAB.stop == 1].sample(10)

,term_str,n,num,stop
term_id,,,,
9009,his,5663,0,1
1703,been,1564,0,1
20920,we,1695,0,1
1789,below,33,0,1
8764,have,2458,0,1
21423,y,10,0,1
2531,by,2664,0,1
16235,s,3,0,1
21478,yourself,86,0,1


In [34]:
# adding stems to vocab
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.term_str.apply(stemmer1.stem)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.term_str.apply(stemmer2.stem)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.term_str.apply(stemmer3.stem)

In [44]:
LIB

,book_title,book_file
book_id,,
144,"The Voyage Out, by Virginia Woolf",144.txt
145,"Middlemarch, by George Eliot",145.txt
18247,"The Last Man, by Mary Wollstonecraft Shelley",18247.txt


In [45]:
# save
#DOC.to_csv("DOC.csv")
LIB.to_csv("LIB_final.csv")
#VOCAB.to_csv("VOCAB.csv")
#TOKEN.to_csv("TOKEN.csv")

In [17]:
VOCAB = pd.read_csv('VOCAB.csv')

In [35]:
VOCAB.head(20)

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster
term_id,,,,,,,
0,,7301,0,0,,,
1,1,2,1,0,1,1,1
2,10,2,1,0,10,10,10
3,11,1,1,0,11,11,11
4,112,1,1,0,112,112,112
5,12,2,1,0,12,12,12
6,13,1,1,0,13,13,13
7,1348,1,1,0,1348,1348,1348
8,14,2,1,0,14,14,14


In [36]:
# find the most frequent POS tag associated with a specific term
def get_POS_Count(row):
    term = row['term_str']

    return TOKEN[TOKEN['term_str']== term].value_counts('pos').idxmax() 

In [37]:
def get_POS_Count(row):
    term = row['term_str']
    filtered_tokens = TOKEN[TOKEN['term_str'] == term]
    if not filtered_tokens.empty:
        return filtered_tokens['pos'].value_counts().idxmax()
    else:
        return None  # Handle the case where the sequence is empty

In [38]:
VOCAB['pos_max'] = VOCAB.apply(get_POS_Count, axis=1)

In [39]:
VOCAB.sample(10)

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max
term_id,,,,,,,,
5507,disreputable,3,0,0,disreput,disreput,disreput,JJ
8542,guido,1,0,0,guido,guido,guido,NNP
14356,preferable,3,0,0,prefer,prefer,pref,JJ
2642,canoes,1,0,0,cano,cano,cano,NNS
13698,persuasively,2,0,0,persuas,persuas,persuas,NN
14028,pleasure,133,0,0,pleasur,pleasur,pleas,NN
15111,rap,2,0,0,rap,rap,rap,NN
4120,cornish,1,0,0,cornish,cornish,corn,NNP
13625,perfumed,2,0,0,perfum,perfum,perfum,VBN


In [40]:
# add term rank to VOCAB
if 'term_rank' not in VOCAB.columns:
    VOCAB = VOCAB.sort_values('n', ascending=False).reset_index()
    VOCAB.index.name = 'term_rank'
    VOCAB = VOCAB.reset_index()
    VOCAB = VOCAB.set_index('term_id')
    VOCAB['term_rank'] = VOCAB['term_rank'] + 1

In [41]:
new_rank = VOCAB.n.value_counts()\
    .sort_index(ascending=False).reset_index().reset_index()\
    .rename(columns={'level_0':'term_rank2', 'index':'n', 'n':'nn'})\
    .set_index('n')
VOCAB['term_rank2'] = VOCAB.n.map(new_rank.term_rank2) + 1
VOCAB.head()

,term_rank,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max,term_rank2
term_id,,,,,,,,,,
18871,1,the,26880,0,1,the,the,the,DT,1
12845,2,of,17206,0,1,of,of,of,IN,2
764,3,and,16460,0,1,and,and,and,CC,3
19145,4,to,16159,0,1,to,to,to,TO,4
65,5,a,11162,0,1,a,a,a,DT,5


In [42]:
TOKEN.head()

pos_tuple  pos token_str  \
book_id chap_num para_num sent_num token_num                                  
144     27       1        0        0           (CHAPTER, NN)   NN   CHAPTER   
                                   1                (I, PRP)  PRP         I   
                 2        0        0                (As, IN)   IN        As   
                                   1               (the, DT)   DT       the   
                                   2          (streets, NNS)  NNS   streets   

                                             term_str  
book_id chap_num para_num sent_num token_num           
144     27       1        0        0          chapter  
                                   1                i  
                 2        0        0               as  
                                   1              the  
                                   2          streets

In [43]:
VOCAB.to_csv("VOCAB_final.csv")
TOKEN.to_csv("TOKEN_final.csv")